## Engagement recognition with Keras

In [26]:
from keras.layers import Dense, LSTM, GRU, Dropout, Flatten, Embedding,  BatchNormalization
from keras.layers.convolutional import Conv1D, MaxPooling1D
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import keras
from keras.models import Model,Sequential
from keras.utils import to_categorical
from keras import regularizers
from keras.optimizers import SGD
from clr_callback import *
%matplotlib inline
import time
import pickle
import os
import math
import torch.nn as nn
from torch.nn.modules.utils import _triple
from keras.utils import to_categorical
import tensorflow as tf
from keras.callbacks import EarlyStopping
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report



In [27]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [28]:
SPLIT_METHOD_child = "CHILD"
SPLIT_METHOD_session = "SESSION"
SPLIT_METHOD_random = "RANDOM"

# WITH FACE, BODY, HANDS (411) =>
FEAT_NUM_all = 201

# WITH BODY AND HANDS (201) =>
FEAT_NUM_body = (25 + 21 + 21) * 3

# WITH FACE ONLY (210) =>
FEAT_NUM_face = 70 * 3

SEQ_LENGTH = 1

NB_CLASS = 6
CSV_FILE = "ASD_36_res1.csv"
VARS_DIR = "var"


Load Data

In [29]:
import pandas
data = pandas.read_csv('ASD_36_res1.csv')

Extract data for 25 body keypoints

In [30]:
X = data.iloc[:, 3:78]
y = data['engagement']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.14, random_state=1)

In [31]:
X_train.shape


(253124, 75)

Split data to train, test, validation

In [32]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [33]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(227811, 75)
(25313, 75)
(41207, 75)


In [34]:
X_train = X_train.to_numpy()
X_test =X_test.to_numpy()
X_val = X_val.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_val = y_val.to_numpy()

In [35]:
type(X_train)

numpy.ndarray

In [36]:

early_stop = EarlyStopping(monitor='loss', patience=2)


For conv1d layer input, data should be reshaped into 3 dimentional

In [46]:
## This cell is needed if the first layer of the network is LSTM (Uncomment this cell and comment the cell below)
#xx = tf.convert_to_tensor(X_train.reshape(227811,1 , 75))
#xv = tf.convert_to_tensor(X_val.reshape(25313, 1, 75))
#xt = tf.convert_to_tensor(X_test.reshape(41207,1, 75))
#yy = tf.convert_to_tensor(y_train.reshape(227811, 1))
#yv = tf.convert_to_tensor(y_val.reshape(25313, 1))
#yt = tf.convert_to_tensor(y_test.reshape(41207, 1))

In [37]:
xx = tf.convert_to_tensor(X_train.reshape(227811,75 , 1))
xv = tf.convert_to_tensor(X_val.reshape(25313, 75, 1))
xt = tf.convert_to_tensor(X_test.reshape(41207, 75, 1))
yy = tf.convert_to_tensor(y_train.reshape(227811, 1))
yv = tf.convert_to_tensor(y_val.reshape(25313, 1))
yt = tf.convert_to_tensor(y_test.reshape(41207, 1))

We have 6 classes and it is better to convert y which is one dimentional with values (0,1,2,3,4,5) to 6 dimentional [0 1 0 0 0 0] -> indicates 1

In [38]:
yy= to_categorical(yy)
yv =  to_categorical(yv)
yt = to_categorical(yt)

Keras sequential model

In [52]:
model = keras.Sequential()

model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(75,1)))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))

model.add(Dropout(0.2))

model.add(MaxPooling1D(pool_size=1))

model.add(Flatten())

model.add(keras.layers.Dense(128, activation='relu'))

model.add(keras.layers.Dense(256, activation='relu'))

model.add(keras.layers.Dense(128, activation='relu'))

model.add(keras.layers.Dense(64, activation='relu'))

model.add(keras.layers.Dense(6, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_35 (Conv1D)           (None, 73, 64)            256       
_________________________________________________________________
conv1d_36 (Conv1D)           (None, 71, 64)            12352     
_________________________________________________________________
conv1d_37 (Conv1D)           (None, 69, 32)            6176      
_________________________________________________________________
conv1d_38 (Conv1D)           (None, 67, 32)            3104      
_________________________________________________________________
conv1d_39 (Conv1D)           (None, 65, 32)            3104      
_________________________________________________________________
dropout_7 (Dropout)          (None, 65, 32)            0         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 65, 32)           

NUmber of batch size and epochs (also can be tuned)

In [53]:
hist = model.fit(x = xx , y = yy, epochs=100, validation_data =(xv, yv), batch_size= 256)


Epoch 1/100
890/890 [==============================] - 6s 7ms/step - loss: 1.3806 - accuracy: 0.4712 - val_loss: 1.3025 - val_accuracy: 0.4971
Epoch 2/100
890/890 [==============================] - 6s 6ms/step - loss: 1.2534 - accuracy: 0.5159 - val_loss: 1.2279 - val_accuracy: 0.5268
Epoch 3/100
890/890 [==============================] - 6s 6ms/step - loss: 1.1868 - accuracy: 0.5436 - val_loss: 1.1533 - val_accuracy: 0.5542
Epoch 4/100
890/890 [==============================] - 6s 7ms/step - loss: 1.1371 - accuracy: 0.5630 - val_loss: 1.1362 - val_accuracy: 0.5620
Epoch 5/100
890/890 [==============================] - 6s 7ms/step - loss: 1.0961 - accuracy: 0.5807 - val_loss: 1.1029 - val_accuracy: 0.5821
Epoch 6/100
890/890 [==============================] - 6s 7ms/step - loss: 1.0590 - accuracy: 0.5963 - val_loss: 1.0683 - val_accuracy: 0.5944
Epoch 7/100
890/890 [==============================] - 6s 7ms/step - loss: 1.0297 - accuracy: 0.6081 - val_loss: 1.0438 - val_accuracy: 0.6017

Epoch 58/100
890/890 [==============================] - 6s 7ms/step - loss: 0.6522 - accuracy: 0.7584 - val_loss: 0.9494 - val_accuracy: 0.6729
Epoch 59/100
890/890 [==============================] - 6s 7ms/step - loss: 0.6481 - accuracy: 0.7599 - val_loss: 0.9588 - val_accuracy: 0.6774
Epoch 60/100
890/890 [==============================] - 6s 7ms/step - loss: 0.6440 - accuracy: 0.7626 - val_loss: 0.9608 - val_accuracy: 0.6765
Epoch 61/100
890/890 [==============================] - 6s 7ms/step - loss: 0.6416 - accuracy: 0.7628 - val_loss: 0.9465 - val_accuracy: 0.6793
Epoch 62/100
890/890 [==============================] - 6s 7ms/step - loss: 0.6391 - accuracy: 0.7641 - val_loss: 0.9444 - val_accuracy: 0.6793
Epoch 63/100
890/890 [==============================] - 6s 7ms/step - loss: 0.6386 - accuracy: 0.7642 - val_loss: 0.9466 - val_accuracy: 0.6783
Epoch 64/100
890/890 [==============================] - 6s 7ms/step - loss: 0.6353 - accuracy: 0.7645 - val_loss: 0.9455 - val_accuracy:

In [54]:
h = model.evaluate(xt,yt)

1288/1288 [==============================] - 2s 1ms/step - loss: 0.9368 - accuracy: 0.6920
